In [1]:
import sys
sys.path.append('../script/')
import os
from os.path import exists
import gc
from functools import partial

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import optuna

import utils
import models
import train as trainer
DEVICE = "cuda"
EPOCHS = 3
MODELNAME = "Baseline1013"
if not exists(MODELNAME):
    os.makedirs(f"{MODELNAME}/tensorboard")
save_model = False
writer = SummaryWriter(log_dir=f"{MODELNAME}/tensorboard")

In [2]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")

In [3]:
fold = 0
print(f'[Fold No.{fold:>3}]\n')
train_df = df[df.kfold != fold].reset_index(drop=True)
valid_df = df[df.kfold == fold].reset_index(drop=True)

[Fold No.  0]



In [4]:
x_tr = train_df[features].to_numpy()
x_va = valid_df[features].to_numpy()

In [5]:
y_tr = train_df[targets].to_numpy()
y_va = valid_df[targets].to_numpy()

In [6]:
params = {
    "nn_params": {"dropout": 0.2, "num_layers": 3, "hidden_size": 256, "activation": "relu", "batchnorm": True},
    "optimizer": "SGD",
    "optim_params": {"lr": 1e-2, "momentum": 0.0},
    "scheduler": "ReduceLROnPlateau",
    "scdl_params": {"threshold": 0.00001},
    "batch_size": 256,
}

In [7]:
dataset_tr = utils.MoaDataset(x_tr, y_tr)
loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=params['batch_size'], num_workers=2)
dataset_va = utils.MoaDataset(x_va, y_va)
loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=params['batch_size'], num_workers=2)

In [8]:
class BaseLine(nn.Module):
    def __init__(self, num_features, num_targets, num_layers=3, dropout=.2, hidden_size=256, activation="relu", batchnorm=True):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.Linear(num_features if len(layers)==0 else hidden_size, hidden_size, bias=(not batchnorm)))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            if activation == "relu":
                layers.append(nn.ReLU())
            elif activation == "prelu":
                layers.append(nn.PReLU())
            else:
                raise RuntimeError(f'{activation} is not implemented')
        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [9]:
model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
model.to(DEVICE)

BaseLine(
  (model): Sequential(
    (0): Linear(in_features=879, out_features=256, bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=False)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.2, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=256, bias=False)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Dropout(p=0.2, inplace=False)
    (11): ReLU()
    (12): Linear(in_features=256, out_features=207, bias=True)
  )
)

In [10]:
if params["optimizer"] == "SGD":
    optimizer = torch.optim.SGD(model.parameters(), **params["optim_params"])
elif params["optimizer"] == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), **params["optim_params"])
else:
    raise RuntimeError(f'{params["optimizer"]} is not implemented')

In [11]:
if params["scheduler"] == "ReduceLROnPlateau":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, mode="min", **params["scdl_params"])
else: 
    print("Not Implemented: No scheduling will be applied")
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda ep: 1**ep)

In [12]:
eng = utils.Engine(model, optimizer, device=DEVICE)

del df, train_df, valid_df, x_tr, x_va, y_tr, y_va
gc.collect()

88

In [13]:
loss_best = np.inf
patience = 10
patience_cnt = 0
for ep in range(EPOCHS):
    loss_tr = eng.train(loader_tr)
    loss_va = eng.validate(loader_va)
    scheduler.step(loss_va)
    writer.add_scalar('loss/train', loss_tr, ep)
    writer.add_scalar('loss/valid', loss_va, ep)
    if loss_va < loss_best:
        loss_best = loss_va
        if save_model:
            pass
    else:
        patience_cnt += 1
    if patience_cnt > patience:
        break

In [14]:
filename = f"{MODELNAME}/fold{fold}.pt"
torch.save(model.model.state_dict(), filename)
print("model saved at:", filename)

model saved at: Baseline1013/fold0.pt


In [15]:
writer.close()